# 一、探索数据
## 1、读取数据
* `read_data(name=None, source_path=None)`

## 2、查看数据类型

## 3、数据缺失及唯一值确定

In [ ]:
# 读取数据
def read_data(name=None, source_path=None):
    if name is None:
        print('Source data NA. GG')
    elif name == 'dsId':
        return ds.load(source_path)
    elif name == 'df':
        return pd.read_csv(source_path, sep='\t')
    return 0

In [ ]:
# 查看数据维度
print(f'Train dataset has {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'Test dataset has {test.shape[0]} rows and {test.shape[1]} columns.')
# 查看哪些列具有缺失值
print(f'There are {train_transaction.isnull().any().sum()} columns in train dataset with missing values.')
print(f'There are {train_identity.isnull().any().sum()} columns in train dataset with missing values.')
# 查看数据值唯一的列
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
print(one_value_cols == one_value_cols_test)
print(f'There are {len(one_value_cols)} columns in train dataset with one unique value.')
print(f'There are {len(one_value_cols_test)} columns in test dataset with one unique value.')
# 查看数据缺失值情况
nan_clos = [col for col in rawData.columns if rawData[col].isna().sum() / rawData.shape[0] > 0.90]
print(f'There are {len(nan_clos)} columns in train dataset with ... value.')
drop_cols = nan_clos + one_value_cols

In [ ]:
# 查看数据类型
train.dtypes
# 查看缺失值并排序
train.isna().sum().sort_values()
# 删除缺失值大于95%
rawData.columns[(rawData.isna().sum() / rawData.shape[0])>0.89]
# 查看特征类型 and 判断类别型
train.nunique().sort_values()

# 一、验证训练集和测试集的分布是否相同
1. 训练集 = 1 / 测试集 = 0
2. 二分类模型训练
3. 如果AUC=0.5左右可以用，否则需要删除噪声

In [ ]:
# 训练集和测试集的数据是否不同
# 训练集 测试集 打标签
train_data['u_label'] = 0
test_data['u_label'] = 1
train_new_data = train_data.drop(columns=['label'])
valid_data = pd.concat([train_new_data, test_data], axis=0)
valid_new_data = valid_data.drop(columns=['tagid', 'time'])
# 简单特征处理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
valid_new_data['provice'] = le.fit_transform(valid_new_data.provice)
valid_new_data['city'] = le.fit_transform(valid_new_data.city)
valid_new_data['model'] = le.fit_transform(valid_new_data.model)
valid_new_data['make'] = le.fit_transform(valid_new_data.make)
# 划分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(valid_new_data.iloc[:,0:-1], valid_new_data.iloc[:,-1], test_size=0.3)
# 模型训练
import lightgbm as lgb
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc'
}
train_set = lgb.Dataset(x_train, y_train)
valid_set = lgb.Dataset(x_test, y_test)
lgb.train(params=params, train_set=train_set, valid_sets=[train_set, valid_set], num_boost_round=1000, verbose_eval=100)